In [1]:
import pymongo
from pymongo import MongoClient

import numpy as np
import pandas as pd
import pickle
from pprint import pprint
from IPython.display import clear_output

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["github_projects"]

In [4]:
!mongorestore -d github_usernames github_usernames_dump  #  clean_ready_users

db.list_collection_names()

2019-03-22T12:14:13.687-0700	the --db and --collection args should only be used when restoring from a BSON file. Other uses are deprecated and will not exist in the future; use --nsInclude instead
2019-03-22T12:14:13.687-0700	building a list of collections to restore from github_usernames_dump dir
2019-03-22T12:14:13.688-0700	don't know what to do with file "github_usernames_dump/.DS_Store", skipping...
2019-03-22T12:14:13.688-0700	don't know what to do with subdirectory "github_usernames_dump/github_usernames", skipping...
2019-03-22T12:14:13.688-0700	done


['users']

In [5]:
collection = db.get_collection('users')
cursor = db.users.find({ "following":{"$exists":True }, "followers":{"$exists" : True}}) # get users who have both following list and followers

In [6]:
count = cursor.count()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


In [7]:
print(count)

50295


In [8]:
# #cursor = collection.find({})
# for document in cursor: 
#     print(document['username']) #document['following'])# (

In [9]:
!mongodump -d github_usernames -o clean_ready_users

2019-03-22T12:14:15.893-0700	writing github_usernames.users to 
2019-03-22T12:14:15.965-0700	done dumping github_usernames.users (11809 documents)


In [10]:
df1 = pd.DataFrame(list(cursor))

In [11]:
#pprint(df1.head())

In [12]:
df2 = df1[['username','following']].copy()
df2['rating'] = 1

In [13]:
df2.head(20)

,username,following,rating
0,donnemartin,"[mitsuhiko, kenneth-reitz, wesm, jakevdp]",1
1,kennethreitz,[],1
2,google,[],1
3,tensorflow,[],1
4,pallets,[],1
5,django,[],1
6,mitsuhiko,"[brosner, alex, ericflo, bryanveloso, jezdez, ...",1
7,jakubroztocil,"[mojombo, defunkt, roncohen, alex, jashkenas, ...",1
8,ansible,[],1
9,nvbn,"[vgarvardt, hdhog, dtantsur, zcho, chemikadze,...",1


In [14]:
df3 = pd.DataFrame(df2.following.tolist(), index=df2.username).stack().reset_index(level=1, drop=False) # stacking by username with followers
df3.head()

,level_1,0
username,,
donnemartin,0,mitsuhiko
donnemartin,1,kenneth-reitz
donnemartin,2,wesm
donnemartin,3,jakevdp
mitsuhiko,0,brosner


In [15]:
df3.rename({0:"following"},axis=1, inplace=True)

In [16]:
df3.rename({"level_1":"rating"},axis=1, inplace=True)

In [17]:
df3['rating']=1
df3.head()

,rating,following
username,,
donnemartin,1,mitsuhiko
donnemartin,1,kenneth-reitz
donnemartin,1,wesm
donnemartin,1,jakevdp
mitsuhiko,1,brosner


In [18]:
df4 = df3.reset_index(level=0, drop=False)

In [19]:
df4.head()

,username,rating,following
0,donnemartin,1,mitsuhiko
1,donnemartin,1,kenneth-reitz
2,donnemartin,1,wesm
3,donnemartin,1,jakevdp
4,mitsuhiko,1,brosner


In [20]:
with open("data/df4.pickle", 'wb') as fp:
    pickle.dump(df4, fp)

# shape to % of data, (move to aws machine to) run surprise

In [21]:
new_df = df2
# new_df = new_df.replace(to_replace='', value=np.nan).dropna() # works when space delimeted string in followers col
new_df = new_df[new_df.astype(str)['following'] != '[]'] # works on strings in list
new_df.head(20)
#new_df.shape

,username,following,rating
0,donnemartin,"[mitsuhiko, kenneth-reitz, wesm, jakevdp]",1
6,mitsuhiko,"[brosner, alex, ericflo, bryanveloso, jezdez, ...",1
7,jakubroztocil,"[mojombo, defunkt, roncohen, alex, jashkenas, ...",1
9,nvbn,"[vgarvardt, hdhog, dtantsur, zcho, chemikadze,...",1
15,josephmisiti,"[mojombo, defunkt, vanpelt, macournoyer, ry, b...",1
19,jkbrzt,[jakubroztocil],1
21,minimaxir,[Stitchpunk],1
35,eriklindernoren,"[karpathy, alexjc, fchollet, llSourcell, junya...",1
36,wangshub,"[kenneth-reitz, paulyang1990, dongweiming, lil...",1
39,soimort,"[josh, maccman, jgm, aaronsw, sethladd, philc,...",1


In [22]:
# create col to sort by num following to get most relevant users
new_df['follow_num'] = new_df['following'].str.len() #len(text.split(','))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
new_df.head(20)

In [23]:
new_df = new_df.sort_values('follow_num', ascending=False)
new_df.head(20)

,username,following,rating,follow_num
50294,nishowsan,"[josh, benschwarz, andrew, tmm1, mitsuhiko, da...",1,30
43958,juanvelascogomez,"[JJ, fdavidcl, psicobyte, GoldraK, oslugr, fer...",1,30
18334,callmephilip,"[queso, holman, cho45, jashkenas, rauchg, remy...",1,30
18342,riywo,"[mojombo, wycats, macournoyer, mattetti, al3x,...",1,30
18343,ryanj,"[bryanl, jchris, ncdc, mikeal, quirkey, krober...",1,30
18353,shownx,"[tilgovi, goodfeli, mli, piiswrong, marvis, pa...",1,30
18355,jackfirth,"[technomancy, pmatos, zenspider, rntz, zkat, t...",1,30
43967,ana0,"[pjf, wolever, huertanix, wanderer, garrying, ...",1,30
43960,soyn,"[ry, jashkenas, substack, rauchg, pmq20, josha...",1,30
18361,pein0119,"[jinzhu, cortesi, paulirish, agentzh, rspivak,...",1,30


In [24]:
# stack dataframe

stacked_new_df = pd.DataFrame(new_df.following.tolist(), index=new_df.username).stack().reset_index(level=1, drop=False)
stacked_new_df.head()

,level_1,0
username,,
nishowsan,0,josh
nishowsan,1,benschwarz
nishowsan,2,andrew
nishowsan,3,tmm1
nishowsan,4,mitsuhiko


In [25]:
# fix axis names
stacked_new_df.rename({0:"following"},axis=1, inplace=True)
stacked_new_df.rename({"level_1":"rating"},axis=1, inplace=True)

In [26]:
stacked_new_df.head()

,rating,following
username,,
nishowsan,0,josh
nishowsan,1,benschwarz
nishowsan,2,andrew
nishowsan,3,tmm1
nishowsan,4,mitsuhiko


In [31]:
# fix levels after stack
stacked_new_df = stacked_new_df.reset_index(level=0, drop=False)


In [ ]:
stacked_new_df.drop('index', inplace=True, axis = 1)

In [32]:
stacked_new_df.head()

,level_0,username,rating,following
0,0,nishowsan,0,josh
1,1,nishowsan,1,benschwarz
2,2,nishowsan,2,andrew
3,3,nishowsan,3,tmm1
4,4,nishowsan,4,mitsuhiko


In [53]:
# reset rating col, remove level col
stacked_new_df['rating']=1
stacked_new_df.drop('level_0', inplace=True, axis = 1)
#stacked_new_df.head()

# now stacked_new_df is a clean stacked working copy ordered by number of people user is following

In [42]:
len(stacked_new_df['username'].unique())

30728

In [43]:
stacked_new_df.shape

(435472, 3)

In [46]:
# add in negative rating info

stacked_new_df2 = pd.DataFrame()

for username in stacked_new_df['username'].unique():
    #print(username)
    stacked_new_df2 = stacked_new_df2.append({'username': username, 'rating': int(0), 'following': 'alexpfox'}, ignore_index=True)
    

# for username in stacked_new_df2['username'].unique():
#     # print(username)
#     # if they do actually follow josh, don't add him
#         stacked_new_df2 = stacked_new_df2.append({'username':username, 'rating':0, 'following': 'alexpfox'}, ignore_index=True)
#         continue
#     else:
#         #stacked_new_df2 = stacked_new_df2.append({'username':username, 'rating':0, 'following': 'josh'}, ignore_index=True)

In [48]:
stacked_zeros_df = pd.DataFrame()
stacked_zeros_df.shape

(0, 0)

In [49]:
stacked_zeros_df = stacked_new_df.append(stacked_new_df2)

stacked_zeros_df.head()

,following,rating,username
0,josh,1.0,nishowsan
1,benschwarz,1.0,nishowsan
2,andrew,1.0,nishowsan
3,tmm1,1.0,nishowsan
4,mitsuhiko,1.0,nishowsan


In [50]:
stacked_zeros_df.shape

(466200, 3)

In [ ]:
#stacked_new_df2.loc[stacked_new_df2['rating'] == 0] # works!

In [51]:
with open("data/stacked_zeros_df.pickle", 'wb') as fp:
    pickle.dump(stacked_zeros_df, fp)

In [52]:
stacked_zeros_df.shape

(466200, 3)

## reshape df

In [ ]:
df2.head()

In [ ]:
df5 = df2

# print(df5.head)

for username in range(0, len(df2)):
    df5.at[username, 'following'] = ' '.join(df5.at[username, 'following'])
    
df5.head(20)    

In [ ]:
df5.shape

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

In [ ]:
cv = CountVectorizer()

In [ ]:
cv1 = cv.fit_transform(df2.following) # creates dense matrix

In [ ]:
print(cv1)

In [ ]:
cv2 = cv1.toarray()

In [ ]:
print(cv2)

In [ ]:
with open("data/df.pickle", 'wb') as fp:
    pickle.dump(df5, fp)

## Build a recommender

In [ ]:
# happening in aws

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

#dfs['rating'] = 5
#df.head()

reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = cv2 #Dataset.load_from_df(dfs[['username', 'following', 'rating']], reader)

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(NormalPredictor(), data, cv=2)